In [1]:
import pandas as pd
import numpy as np

# 可視化用のライブラリ
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

import neologdn
import MeCab

import re

from tqdm import tqdm_notebook as tqdm
from tqdm._tqdm_notebook import tqdm_notebook
# # プログレスバーのラベル設定
tqdm_notebook.pandas(desc="progress: ")

In [2]:
df = pd.read_pickle('posi_nega_alldata.pkl')
df.head()

,text,posi_nega
0,案件個別の技術相談乗ってます！乗ります,1
1,勉強とお仕事が有機的に繋がっていて面白い。,1
2,次々とテクニックを教えていただき、ためになりました,1
3,お肉がキレイに焼けたことに小さなしあわせを感じた,1
4,これが私の最適化,1


## 単語を形態素解析しリストに格納、リストに格納した単語をスペースで区切り文字列型に変換する関数を定義

In [3]:
'''単語を形態素解析しリストに格納、リストに格納した単語をスペースで区切り文字列型に変換する関数'''
def get_mecabed_words_text(text):
    if not isinstance(text, str):
        pass
    else:
        sentence_normalization = neologdn.normalize(text)
        neologd_tagger = MeCab.Tagger('-Ochasen -d /usr/local/lib/mecab/dic/mecab-ipadic-neologd')
        # neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して表示される
        # 原形、品詞などの間には「\t」が、分かち書きされた単語と単語の区切りには「\n」が表示される
        # 例: '空い\tアイ\t空く\t動詞-自立\t五段・カ行イ音便\t連用タ接続\nた\tタ\tた\t助動詞\t特殊・タ\t基本形\n時間....
        # まずはparse()で分かち書きした単語群は1つの文字列型になっているので「\n」で区切り、リスト型にする
        wakati_text_list = neologd_tagger.parse(sentence_normalization).split('\n')
        # 「\n」で区切り、リスト型にした結果の例は下記
        # ['空い\tアイ\t空く\t動詞-自立\t五段・カ行イ音便\t連用タ接続', ：リスト0番目
        # 'た\tタ\tた\t助動詞\t特殊・タ\t基本形',：リスト1番目
        # '時間\tジカン\t時間\t名詞-副詞可能\t\t',：リスト2番目

        original_form_list =[]# 単語の「原形」のみ格納する(品詞情報は単語頻度を求める際に必要ないので除外)

        # parse() の出力結果の最後は「EOS」という文字のみ
        # EOSのとき、pos = wakati.split('\t')[3]の要素はないので下記forループを実行すると「list index out of range」とエラーを発生させてしまう
        # よってEOSのときは条件分岐if~breakでforループから抜け出すよう記述
        for wakati in wakati_text_list:
            surface = wakati.split('\t')[0]
            if surface == 'EOS':
                break
            else:
                word = wakati.split('\t')[2]
                original_form_list.append(word)

        return ' '.join(original_form_list)

## 全文章に対し形態素解析、文章毎に出力された単語をスペースで区切り、格納
上記関数を実行する

In [4]:
# [text]カラムの全行に対してget_mecabed_words_text関数を実行
# 文章毎に形態素解析結果が[案件 個別 の 技術]　といった単語間のスペース入りの文字列形式で格納される
# 今回apply関数をSeries型（text_df['text']）に対して使用するためaxis=1を引数に与えない
df['wakati'] = df['text'].progress_apply(get_mecabed_words_text)
df.head()

,text,posi_nega,wakati
0,案件個別の技術相談乗ってます！乗ります,1,案件 個別 の 技術 相談 乗る て ます ! 乗る ます
1,勉強とお仕事が有機的に繋がっていて面白い。,1,勉強 と お仕事 が 有機的 に 繋がる て いる て 面白い 。
2,次々とテクニックを教えていただき、ためになりました,1,次々 と テクニック を 教える て いただく 、 ため に なる ます た
3,お肉がキレイに焼けたことに小さなしあわせを感じた,1,お肉。 が キレイ に 焼ける た こと に 小さな しあわせ を 感じる た
4,これが私の最適化,1,これ が 私 の 最適化
